### 🧠 Seção 6: O Ajuste de Curvas Clássico com `scipy`

Agora que entendemos o "quê" e o "porquê", vamos ao "como". Nesta seção, vamos usar a principal ferramenta do Python para ajuste de curvas, `scipy.optimize.curve_fit`, para encontrar os parâmetros de um modelo teórico que descreve um fenômeno de crescimento.

#### 6.1. Cenário: Modelando Crescimento com Saturação (Curva Logística)

Muitos processos na natureza não crescem indefinidamente. Eles começam devagar, aceleram e depois desaceleram à medida que se aproximam de um limite máximo (saturação). Pense na proliferação de uma cultura de bactérias em uma placa de Petri, na disseminação de uma notícia em uma rede social ou na adoção de uma nova tecnologia no mercado.

Este processo é frequentemente descrito pela **função logística** ou **curva sigmoidal**:

$$ y(x) = \frac{L}{1 + e^{-k(x-x_0)}} $$

Diferente dos modelos da Parte I, aqui os parâmetros têm uma interpretação física direta:
* **$L$**: O **limite máximo** da curva (a "capacidade de carga" ou o valor de saturação).
* **$x_0$**: O **ponto médio** da curva (o ponto no eixo X onde a curva atinge $L/2$ e o crescimento é mais rápido).
* **$k$**: A **taxa de crescimento** ou a "inclinação" da curva.

Nosso objetivo é usar um conjunto de dados "experimentais" para estimar os valores verdadeiros de $L, k$ e $x_0$.

Primeiro, vamos definir e gerar os dados para este cenário.

In [1]:
# Importar a biblioteca de otimização do SciPy
from scipy.optimize import curve_fit

# 1. Definir a função do nosso modelo teórico
def modelo_logistico(x, L, k, x0):
    """
    Função logística com parâmetros:
    L: Valor máximo
    k: Taxa de crescimento
    x0: Ponto médio
    """
    return L / (1 + np.exp(-k * (x - x0)))

# 2. Definir os parâmetros "reais" para gerar nossos dados
L_real = 120.0  # Limite máximo de 120 unidades
k_real = 0.2    # Taxa de crescimento
x0_real = 50.0  # O crescimento atinge o pico no tempo x=50

# Gerar o eixo X (ex: 100 dias de observação)
x_data = np.linspace(0, 100, 100)

# Calcular os valores de Y usando o modelo e os parâmetros reais
y_real = modelo_logistico(x_data, L_real, k_real, x0_real)

# Adicionar ruído para simular dados experimentais
ruido = np.random.normal(0, 3.5, size=x_data.size)
y_data = y_real + ruido

# 3. Visualizar nossos dados "experimentais"
plt.figure(figsize=(12, 7))
sns.scatterplot(x=x_data, y=y_data, label='Dados Experimentais')
plt.title('Dados de Crescimento Logístico com Ruído', fontsize=16)
plt.xlabel('Tempo (dias)', fontsize=12)
plt.ylabel('População / Vendas', fontsize=12)
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'np' is not defined

#### 6.2. Ajuste com `curve_fit` e a Importância das Estimativas Iniciais (`p0`)

O algoritmo por trás do `curve_fit` é um otimizador que precisa de um ponto de partida para começar sua busca pelos melhores parâmetros. Para funções simples, ele geralmente se vira bem sozinho. Mas para modelos com múltiplos parâmetros, como o nosso, um "chute" inicial ruim pode fazer com que o algoritmo não encontre a solução correta (não "convirja").

Felizmente, podemos guiar o otimizador fornecendo estimativas iniciais (`p0`), que podemos extrair simplesmente olhando para o gráfico:
* **Estimativa para `L`**: O valor máximo que os dados atingem. Olhando o gráfico, parece ser em torno de 120.
* **Estimativa para `x0`**: O valor de `x` onde a curva parece estar na metade da sua altura máxima (metade de 120, ou 60). Olhando o gráfico, isso acontece perto de `x=50`.
* **Estimativa para `k`**: Esta é a mais difícil de estimar visualmente, mas ela representa a inclinação. Um valor pequeno e positivo como `0.1` ou `0.2` é um bom chute inicial.

Vamos fornecer essas dicas ao `curve_fit` para garantir um bom resultado.

In [ ]:
# 1. Definir nossas estimativas iniciais (p0)
#          [L,   k,   x0]
p0 = [120.0, 0.2, 50.0]

# 2. Usar curve_fit com as estimativas iniciais
# popt: contém os parâmetros ótimos encontrados
# pcov: contém a matriz de covariância, que usaremos para estimar o erro
popt, pcov = curve_fit(modelo_logistico, x_data, y_data, p0=p0)

# 3. Extrair os parâmetros e seus erros
L_est, k_est, x0_est = popt
erros = np.sqrt(np.diag(pcov))
L_err, k_err, x0_err = erros

# 4. Apresentar os resultados de forma clara
print("--- Resultados do Ajuste Não-Linear ---")
print(f"Parâmetro L (Limite Máximo): {L_est:.2f} ± {L_err:.2f} (Real: {L_real})")
print(f"Parâmetro k (Taxa de Crescimento): {k_est:.3f} ± {k_err:.3f} (Real: {k_real})")
print(f"Parâmetro x0 (Ponto Médio): {x0_est:.2f} ± {x0_err:.2f} (Real: {x0_real})")
print("\nOs valores estimados são muito próximos dos valores reais!")

# 5. Visualizar o ajuste final
plt.figure(figsize=(12, 7))
sns.scatterplot(x=x_data, y=y_data, label='Dados Experimentais', alpha=0.7)
plt.plot(x_data, modelo_logistico(x_data, *popt), color='red', linewidth=3, label='Curva Ajustada')
plt.title('Ajuste do Modelo Logístico aos Dados', fontsize=16)
plt.xlabel('Tempo (dias)', fontsize=12)
plt.ylabel('População / Vendas', fontsize=12)
plt.legend()
plt.grid(True)
plt.show()